In [1]:
from google.colab import files

uploaded = files.upload()

uploaded.keys()

Saving stance_examples1.csv to stance_examples1 (1).csv


dict_keys(['stance_examples1 (1).csv'])

In [2]:
!pip install -U transformers

In [3]:
!pip install -q transformers datasets scikit-learn huggingface_hub

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('stance_examples1.csv', encoding='latin1')  # columns: sentence, stance, topic

label_map = {'con':0, 'pro':1}
df['label'] = df['stance'].map(label_map)

# topic-aware input
def make_input(row):
    return f"[TOPIC] {row['topic']} [SEP] {row['sentence']}"
df['text'] = df.apply(make_input, axis=1)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)


In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset

# Define model_name and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create tokenized datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

tokenized_train_ds = train_dataset.map(tokenize_function, batched=True)
tokenized_val_ds = val_dataset.map(tokenize_function, batched=True)

# Select and rename columns for Trainer
train_ds = tokenized_train_ds.remove_columns(['sentence', 'stance', 'topic', '__index_level_0__', 'text'])
val_ds = tokenized_val_ds.remove_columns(['sentence', 'stance', 'topic', '__index_level_0__', 'text'])
train_ds = train_ds.rename_column('label', 'labels')
val_ds = val_ds.rename_column('label', 'labels')
train_ds.set_format('torch')
val_ds.set_format('torch')

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1_macro': f1_score(labels, preds, average='macro')
    }

args = TrainingArguments(
    output_dir='outputs',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1_macro',
    greater_is_better=True,
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Map:   0%|          | 0/215 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4261180725.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [9]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer

# 1. 모델 초기화 (num_labels 지정)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2   # 실제 클래스 수로 변경
)

# 2. 최신 tokenizer 사용
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 3. TrainingArguments에서 W&B 끄기
args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs"
)


# 4. Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 5. 학습 실행
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1423222009.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: skywindstory (skywindstory-seoul) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=42, training_loss=0.704056876046317, metrics={'train_runtime': 1334.8611, 'train_samples_per_second': 0.483, 'train_steps_per_second': 0.031, 'total_flos': 42426657676800.0, 'train_loss': 0.704056876046317, 'epoch': 3.0})

In [12]:
# local save
trainer.save_model('weights/stance-distilbert')
tokenizer.save_pretrained('weights/stance-distilbert')

# Google Drive (Colab)
from google.colab import drive
drive.mount('/content/drive')
!cp -r weights/stance-distilbert /content/drive/MyDrive/stance-distilbert

Mounted at /content/drive
